In [21]:
import cv2
image = cv2.imread("./images/img1.png", 1)
image.shape

(1080, 1440, 3)

In [22]:
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
ret, corners = cv2.findChessboardCorners(gray, (24, 17), None)

In [23]:
print(ret, corners.shape)
corners[0]

True (408, 1, 2)


array([[394.29108, 791.2645 ]], dtype=float32)

In [24]:
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
exact_corners = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)

cv2.drawChessboardCorners(image, (24, 17), exact_corners, ret)
cv2.imshow('Found Corners', image[700:])
cv2.waitKey()

-1

In [25]:
import numpy as np
objp = np.zeros((24*17,3), np.float32)
print(objp.shape)
objp[:,:2] = np.mgrid[0:24,0:17].T.reshape(-1,2)
print(objp)
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera([objp], [corners], gray.shape[::-1], None, None)
ret, mtx, dist, rvecs, tvecs

(408, 3)
[[ 0.  0.  0.]
 [ 1.  0.  0.]
 [ 2.  0.  0.]
 ...
 [21. 16.  0.]
 [22. 16.  0.]
 [23. 16.  0.]]


(0.22975378411606748,
 array([[1.00520531e+03, 0.00000000e+00, 7.91264352e+02],
        [0.00000000e+00, 1.01544875e+03, 5.20980899e+02],
        [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]]),
 array([[-0.20625473,  0.19895327, -0.0008226 , -0.0164914 , -0.18016538]]),
 (array([[-0.02364562],
         [ 0.03777229],
         [-0.00948411]]),),
 (array([[-24.33164253],
         [ 16.51999771],
         [ 60.43367431]]),))

In [26]:
k1, k2, p1, p2, k3 = dist[0]
print(f"k1: {k1}\nk2: {k2}\np1: {p1}\np2: {p2}\nk3: {k3}")

k1: -0.2062547319761049
k2: 0.19895327270447333
p1: -0.0008226027151708499
p2: -0.01649139837134075
k3: -0.1801653765018224


In [31]:
image5 = cv2.imread("./images/img5.png", 1)
w_h = image5.shape[1::-1]
newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, w_h, 1, w_h)

# undistort
dst = cv2.undistort(image5, mtx, dist, None, newcameramtx)

# crop the image
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]
cv2.imwrite('calibresult1.png', dst)

True

In [37]:
images = tuple(cv2.imread(f"./images/img{i}.png", 1) for i in range(1, 4 + 1))
grays = tuple(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) for image in images)
sizes = ((24, 17), (17, 24), (24, 17), (24, 17))
corners_list = tuple(cv2.findChessboardCorners(gray, size, None)[1] for gray, size in zip(grays, sizes))

objpoints = []
for size in sizes:
    objp = np.zeros((24*17,3), np.float32)
    objp[:,:2] = np.mgrid[0:size[0],0:size[1]].T.reshape(-1,2)
    objpoints.append(objp)

ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, corners_list, gray.shape[::-1], None, None)

newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, w_h, 1, w_h)

# undistort
dst = cv2.undistort(image5, mtx, dist, None, newcameramtx)

# crop the image
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]
cv2.imwrite('calibresult2.png', dst)

True